### Import libraries

In [31]:
import numpy as np
import pandas as pd
import gurobipy as grb
from gurobipy import GRB
import random

### Read and clean initial dataset

In [32]:
data = pd.read_csv("data.csv")
data

,Crs & Sec,CRN,Title,Cred,Days,Times,Room,Instructor
0,AFR-101-0,10001.0,Intro to Africana Studies,1.0,TR,9:40 - 10:55 AM,LIB STUDIOD,Garcia-Rojas C
1,AFR-283-0,10529.0,"Islands, Arch & Blk Womns Lit",1.0,TR,1:40 - 2:55 PM,LIB STUDIOD,Gill-Sadler R
2,AFR-302-0,10540.0,Blk Wmn's Intellectual History,1.0,TR,8:15 - 9:30 AM,CHAM 1045,Green H
3,AFR-495-0,10514.0,Africana Capstone,1.0,W,1:30 - 4:20 PM,CHAM 1046,Green H
4,ANT-101-0,10003.0,Intro Cultural Anthropology,1.0,TR,9:40 - 10:55 AM,WATSON 147,Shuman S
...,...,...,...,...,...,...,...,...
461,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
462,NaN,NaN,NaN,NaN,TR,9:40 - 10:55 AM,WALL B46,NaN
463,PSY-302-D,10508.0,Psy Research: Behavior Pharm,1.0,MWF,8:30 - 9:20 AM,CHAM 1045,Smith M
464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
data.columns

Index(['Crs & Sec', 'CRN', 'Title', 'Cred', 'Days', 'Times', 'Room',
       'Instructor'],
      dtype='object')

Drop rows containing NAN values

In [34]:
data.dropna(inplace=True)
data.isnull().values.any()
data.reset_index(inplace=True)

#### Get the classrooms data

In [35]:
classrooms_data = data["Room"].unique()
classrooms_data

array(['LIB STUDIOD', 'CHAM 1045', 'CHAM 1046', 'WATSON 147',
       'WATSON 119', 'WATSON 157', 'CHAM 2196', 'WALL 243', 'CHAM 3234',
       'VAC 117', 'VAC 203', 'VAC 223', 'VAC 213', 'VAC 104', 'VAC 105',
       'VAC 106', 'VAC 212', 'VAC 108', 'VAC 214', 'WALL 210', 'WALL 106',
       'LIB B110', 'WALL B05', 'WALL 143', 'CHAM 3130', 'WALL 323',
       'WATSON 109', 'WALL 307', 'WALL 320', 'WALL 351', 'CHAM 1027',
       'CHAM 1015', 'CHAM 3106', 'CHAM 2198', 'CHAM 1086', 'CHAM 1062',
       'CHAM 1003', 'CHAM 1096', 'CHAM B027', 'CHAM 1006', 'WATSON 132',
       'WALL 380', 'CHAM LRC', 'CHAM 3187', 'BAKER 1100', 'CHAM 3198',
       'SLOAN 100', 'CHAM 2164', 'CHAM HANCE', 'WATSON 140', 'DANA 146',
       'CHAM 3155', 'CHAM 2146', 'CHAM 2068', 'CHAM 3084', 'CHAM 2084',
       'CHAM 2187', 'SLOAN 201', 'CHAM 2209', 'CUNN 109', 'EU 101',
       'WATSON 310', 'DANA 220', 'WALL 251', 'CHAM 3068', 'CHAM 2130',
       'CHAM 3209', 'CHAM 2234', 'SLOAN B011', 'SLOAN B020', 'CHAM B022',
     

#### Get the course data

In [36]:
course_data = data[["Crs & Sec", "Instructor"]]
course_data

,Crs & Sec,Instructor
0,AFR-101-0,Garcia-Rojas C
1,AFR-283-0,Gill-Sadler R
2,AFR-302-0,Green H
3,AFR-495-0,Green H
4,ANT-101-0,Shuman S
...,...,...
339,PSY-300-0,Eiler B
340,PSY-302-A,Smith M
341,PSY-302-B,Smith M
342,PSY-302-C,Smith M


#### Get data related to professor
- Professor ID maps to name
- Professor name to course id

In [37]:
prof_data = data["Instructor"].unique()
profs = {i: prof_data[i] for i in range(len(prof_data))}
profs

{0: 'Garcia-Rojas C',
 1: 'Gill-Sadler R',
 2: 'Green H',
 3: 'Shuman S',
 4: 'Cormier A',
 5: 'Bowles L',
 6: 'Joubin R',
 7: 'Martinez T',
 8: 'Starr T',
 9: 'Savage C',
 10: 'St Clair K',
 11: 'Dietrick J',
 12: 'Hundley C',
 13: 'Corso-Esquivel J',
 14: 'Bernd K',
 15: 'Wessner D',
 16: 'Campbell M',
 17: 'Villa S',
 18: 'Levasseur K',
 19: 'Wadgymar S',
 20: 'El Bejjani R',
 21: 'Heyer L',
 22: 'Smith K',
 23: 'Spillman J',
 24: 'Lom B',
 25: 'Meier A',
 26: 'Hales K',
 27: 'Spillman A',
 28: 'Thurtle-Schmidt B',
 29: 'Sarafova S',
 30: 'Sich J',
 31: 'Melonakos J',
 32: 'Alexander C',
 33: 'Striplin D',
 34: 'Fernandez-Torres L',
 35: 'Anstey M',
 36: 'El-Zaatari B',
 37: 'Blauch D',
 38: 'Hauser C',
 39: 'Key H',
 40: 'Stevens E',
 41: 'Myers J',
 42: 'Snyder N',
 43: 'Tsai M',
 44: 'Shao P',
 45: 'Shen V',
 46: 'Ewoodzie J',
 47: 'Krentz P',
 48: 'McClellan A',
 49: 'Truetzel A',
 50: 'Baugh S',
 51: 'Martinez A',
 52: 'Bailey I',
 53: 'Pulaj J',
 54: 'Mendes H',
 55: 'Nemitz C

In [38]:
instr_to_all_course_dict = course_data.groupby('Instructor').apply(lambda x: list(x.index)).to_dict()
instr_to_all_course_dict

{'Ahrensdorf P': [305, 315, 325],
 'Aldridge D': [231, 237],
 'Alexander C': [62, 63],
 'Anstey M': [68, 69, 87],
 'Bahr H': [126, 127],
 'Bailey I': [104, 105, 106],
 'Baker B': [148, 149, 150],
 'Baugh S': [101, 103],
 'Belloni M': [292, 293, 298],
 'Benson R': [169, 170],
 'Berkey J': [227, 238],
 'Bernd K': [29, 30],
 'Blake H': [249, 264],
 'Blauch D': [72, 73],
 'Bond A': [330, 338],
 'Bory A': [124, 125],
 'Botelho M': [265, 268],
 'Bowles L': [8, 9],
 'Boyd M': [331, 336],
 'Boyer P': [241, 242],
 'Breitenfeld S': [247, 248],
 'Bullock G': [308],
 'Bullock S': [273, 274],
 'Busch K': [279],
 'Campbell M': [33, 34],
 'Campbell S': [172],
 'Ceka B': [313, 314, 320],
 'Chan F': [255, 260],
 'Chapman L': [185, 186, 190],
 'Chillag K': [275, 276],
 'Cools A': [140, 141],
 'Cormier A': [5, 7],
 'Corso-Esquivel J': [26, 27],
 'Crandall B': [153, 243],
 'Crandall R': [323],
 'Crowder-Meyer M': [306, 307, 327],
 'Dietrick J': [20, 21, 25],
 'Dietz V': [229, 239],
 'Duhon A': [250, 251, 

In [39]:
course_to_instr_dict = course_data.set_index('Crs & Sec').to_dict()['Instructor']
course_to_instr_dict

{'AFR-101-0': 'Garcia-Rojas C',
 'AFR-283-0': 'Gill-Sadler R',
 'AFR-302-0': 'Green H',
 'AFR-495-0': 'Green H',
 'ANT-101-0': 'Shuman S',
 'ANT-102-0': 'Cormier A',
 'ANT-220-0': 'Shuman S',
 'ANT-340-0': 'Cormier A',
 'ANT-373-0': 'Bowles L',
 'ANT-490-0': 'Bowles L',
 'ARB-101-0': 'Joubin R',
 'ARB-201-0': 'Joubin R',
 'ARB-326-0': 'Joubin R',
 'ART-100-0': 'Martinez T',
 'ART-101-A': 'Starr T',
 'ART-101-B': 'Savage C',
 'ART-103-0': 'St Clair K',
 'ART-105-0': 'Starr T',
 'ART-107-0': 'Starr T',
 'ART-109-0': 'Savage C',
 'ART-111-A': 'Dietrick J',
 'ART-111-B': 'Dietrick J',
 'ART-126-0': 'Hundley C',
 'ART-206-0': 'Hundley C',
 'ART-210-0': 'Martinez T',
 'ART-211-0': 'Dietrick J',
 'ART-218-0': 'Corso-Esquivel J',
 'ART-400-0': 'Corso-Esquivel J',
 'ART-401-0': 'St Clair K',
 'BIO-111-A': 'Bernd K',
 'BIO-111-B': 'Bernd K',
 'BIO-111-C': 'Wessner D',
 'BIO-111-D': 'Wessner D',
 'BIO-113-A': 'Campbell M',
 'BIO-113-B': 'Campbell M',
 'BIO-114-A': 'Villa S',
 'BIO-114-B': 'Villa 

#### Get time period data

In [40]:
time_data = pd.read_csv("class_timeslots.csv")
time_data = time_data.days + " " + time_data.start + "-" + time_data.end
time_data

0       M W F 8:30-9:20
1      M W F 9:30-10:20
2     M W F 10:30-11:20
3     M W F 11:30-12:20
4     M W F 12:30-13:20
5     M W F 13:30-14:20
6     M W F 14:30-15:20
7         M W 8:05-9:20
8       M W 14:30-15:45
9         T R 8:15-9:30
10       T R 9:40-10:55
11      T R 12:15-13:30
12      T R 13:40-14:55
13      T R 15:05-16:20
dtype: object

Pair of overlapping time period

In [41]:
overlap_time = {
    (0,7),
    (6,8)
}

Assign random time preferences to each class since we do not have the real data

In [42]:
n = [i for i in time_data.index]
random.seed(0)
course_data['Preferences'] = course_data.apply(lambda x: random.sample(n, 4), axis=1)
course_data

/var/folders/hl/_9hwm7lx4n1frvtdjpmt8mvr0000gn/T/ipykernel_6414/1913984117.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  course_data['Preferences'] = course_data.apply(lambda x: random.sample(n, 4), axis=1)


,Crs & Sec,Instructor,Preferences
0,AFR-101-0,Garcia-Rojas C,"[13, 6, 12, 0]"
1,AFR-283-0,Gill-Sadler R,"[4, 8, 7, 6]"
2,AFR-302-0,Green H,"[12, 4, 7, 5]"
3,AFR-495-0,Green H,"[9, 3, 8, 2]"
4,ANT-101-0,Shuman S,"[4, 2, 1, 9]"
...,...,...,...
339,PSY-300-0,Eiler B,"[11, 13, 3, 5]"
340,PSY-302-A,Smith M,"[13, 7, 10, 11]"
341,PSY-302-B,Smith M,"[10, 3, 4, 5]"
342,PSY-302-C,Smith M,"[10, 2, 4, 0]"


Get the cardinality of $I , T, C$ 

In [43]:
# total numbers of classes C
I = len(course_data.index)
T = len(time_data.index)
C = len(classrooms_data)
print('Number of classes, time period, classrooms are %s, %s, %s, respectively' % (I, T, C))

Number of classes, time period, classrooms are 344, 14, 77, respectively


### Building Model

#### Model 1: Only need to satisfy the availability condition, no ranking between availabilities
Set Objective Function to Max 0

In [44]:
model = grb.Model()

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-02


In [45]:
x = model.addVars(I, T, C, vtype=GRB.BINARY, name="x")
x[0,0,0]
model.update()

In [46]:
model.setObjective(sense=grb.GRB.MAXIMIZE, expr=0)
model.update()

In [47]:
# Total assigned classroom equals to I
model.addConstr(x.sum() == I, name="Total_assigned_classroom_equals_to_I")

<gurobi.Constr *Awaiting Model Update*>

In [48]:
# Atmost one course per room per time
for t in range(T):
    for c in range(C):
        name = "Atmost_one_course_per_room_per_time_for_time_%s_classroom_%s" % (t, c)
        model.addConstr(x.sum('*', t, c) <= 1, name=name)

In [49]:
# availability of professor for each class
availabities = course_data.Preferences
for i in range(I):
    expr = 0
    avails = availabities[i]
    for avail in avails:
        expr += x.sum(i, avail, '*')
    name = "Availability_constraints_for_course_%s" %i
    model.addConstr(expr == 1, name=name)


In [50]:
# courses taught by the same professor must not happen at the same time or overlapping time
for t in range(T):
    for courses in instr_to_all_course_dict.values():
        expr = 0
        for course in courses:
            expr += x.sum(course, t, '*')
        model.addConstr(expr <= 1, name="Identical_Time_constraint_classes_%s" % (courses))
        
for courses in instr_to_all_course_dict.values():
    for period1, period2 in overlap_time:
        expr = 0
        for course in courses:
            expr = expr + x.sum(course, period1, '*') + x.sum(course, period2, '*')
        model.addConstr(expr <= 1, name="Overlap_Time_constraint_classes_%s_between_time_%s_and_%s" % (courses, period1, period2))

In [51]:
# classrooms must not have overlap time periods
for c in range(C):
    for period1, period2 in overlap_time:
        model.addConstr(x.sum("*", period1, c) + x.sum("*", period2, c) <= 1, name="Overlap_constraint_classroom_%s" % (c))

In [52]:
model.update()

In [53]:
model.optimize()
model.write("Model_1.lp")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4153 rows, 370832 columns and 1430352 nonzeros
Model fingerprint: 0x0c59e6d9
Variable types: 0 continuous, 370832 integer (370832 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [0e+00, 0e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 2640 rows and 268422 columns
Presolve time: 1.06s
Presolved: 1513 rows, 102410 columns, 245245 nonzeros
Variable types: 0 continuous, 102410 integer (102410 binary)
Found heuristic solution: objective -0.0000000

Explored 0 nodes (0 simplex iterations) in 1.22 seconds (2.35 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: -0 
No other solutions better than -0

Optimal solution found (tolerance 1.00e-04)
Best objective -0.000000000000e+00, best bound -0.000000000000e

#### Verify the results and print

In [54]:
if model.Status == GRB.OPTIMAL:
    times = {}
    times_index = {}
    class_rooms = {}
    solution = model.getAttr("X", x)
    for i in range(I):
        for t in range(T):
            for c in range(C):
                if solution[i, t, c] > 0 and t in availabities[i]:
                    times[i] = time_data[t]
                    times_index[i] = t
                    class_rooms[i] = classrooms_data[c]

    assert len(times) == I

In [55]:
res = course_data.copy()
res["Time"] = res.index.map(times)
res["Classroom"] = res.index.map(class_rooms)
res["Time_index"] = res.index.map(times_index)
res["Score"] = res.apply(lambda x: x["Preferences"].index(x["Time_index"]) + 1, axis=1)
res

,Crs & Sec,Instructor,Preferences,Time,Classroom,Time_index,Score
0,AFR-101-0,Garcia-Rojas C,"[13, 6, 12, 0]",T R 15:05-16:20,CHAM 2198,13,1
1,AFR-283-0,Gill-Sadler R,"[4, 8, 7, 6]",M W 8:05-9:20,CHAM 1027,7,3
2,AFR-302-0,Green H,"[12, 4, 7, 5]",T R 13:40-14:55,CUNN 109,12,1
3,AFR-495-0,Green H,"[9, 3, 8, 2]",T R 8:15-9:30,EU 101,9,1
4,ANT-101-0,Shuman S,"[4, 2, 1, 9]",T R 8:15-9:30,CHAM 3084,9,4
...,...,...,...,...,...,...,...
339,PSY-300-0,Eiler B,"[11, 13, 3, 5]",T R 15:05-16:20,VAC 105,13,2
340,PSY-302-A,Smith M,"[13, 7, 10, 11]",T R 15:05-16:20,VAC 213,13,1
341,PSY-302-B,Smith M,"[10, 3, 4, 5]",M W F 11:30-12:20,SLOAN B020,3,2
342,PSY-302-C,Smith M,"[10, 2, 4, 0]",M W F 10:30-11:20,DANA 127,2,2


In [56]:
check = res.groupby("Time")['Instructor'].apply(list).to_dict()
for list_profs in check.values():
    check_set = set()
    for prof in list_profs:
        check_set.add(prof)
    assert len(list_profs) == len(check_set)

In [57]:
total_score = res.Score.sum()
total_score

826

### Model 2: Ranked Availability

In [58]:
model2 = grb.Model()

In [59]:
x = model2.addVars(I, T, C, vtype=GRB.BINARY, name="x")
x[0,0,0]
model2.update()

In [60]:
# availability of professor for each class
availabities = course_data.Preferences
expr = grb.LinExpr()
for i in range(I):
    avails = availabities[i]
    for a in range(len(avails)):
        for c in range(C):
            expr += (a + 1) * x[i, avails[a], c]

model2.setObjective(expr, sense=grb.GRB.MAXIMIZE)
model2.update()



In [61]:
# Total assigned classroom equals to I
model2.addConstr(x.sum() == I, name="Total_assigned_classroom_equals_to_I")

# Atmost one course per room per time
for t in range(T):
    for c in range(C):
        name = "Atmost_one_course_per_room_per_time_for_time_%s_classroom_%s" % (t, c)
        model2.addConstr(x.sum('*', t, c) <= 1, name=name)


for i in range(I):
    expr = grb.LinExpr()
    avails = availabities[i]
    for avail in avails:
        expr += x.sum(i, avail, '*')
    name = "Availability_constraints_for_course_%s" %i
    model2.addConstr(expr == 1, name=name)

# courses taught by the same professor must not happen at the same time or overlapping time
for t in range(T):
    for courses in instr_to_all_course_dict.values():
        expr = grb.LinExpr()
        for course in courses:
            expr += x.sum(course, t, '*')
        model2.addConstr(expr <= 1, name="Identical_Time_constraint_classes_%s" % (courses))
        
for courses in instr_to_all_course_dict.values():
    for period1, period2 in overlap_time:
        expr = grb.LinExpr()
        for course in courses:
            expr = expr + x.sum(course, period1, '*') + x.sum(course, period2, '*')
        model2.addConstr(expr <= 1, name="Overlap_Time_constraint_classes_%s_between_time_%s_and_%s" % (courses, period1, period2))

# classrooms must not have overlap time periods
for c in range(C):
    for period1, period2 in overlap_time:
        model2.addConstr(x.sum("*", period1, c) + x.sum("*", period2, c) <= 1, name="Overlap_constraint_classroom_%s" % (c))

model2.update()

In [62]:
model2.optimize()
model2.write("Model_2.lp")

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[arm])

CPU model: Apple M1 Pro
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 4153 rows, 370832 columns and 1430352 nonzeros
Model fingerprint: 0x7fcdb399
Variable types: 0 continuous, 370832 integer (370832 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 4e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+02]
Presolve removed 2640 rows and 268422 columns
Presolve time: 1.06s
Presolved: 1513 rows, 102410 columns, 245245 nonzeros
Variable types: 0 continuous, 102410 integer (102410 binary)
Found heuristic solution: objective 1149.0000000
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

Use crossover to convert LP symmetric solution to basic solution...

Root relaxation: objective 1.354000e+03, 26861 iterations, 0.10 seconds

In [63]:
if model2.Status == GRB.OPTIMAL:
    times = {}
    times_index = {}
    class_rooms = {}
    solution = model2.getAttr("X", x)
    for i in range(I):
        for t in range(T):
            for c in range(C):
                if solution[i, t, c] > 0 and t in availabities[i]:
                    times[i] = time_data[t]
                    times_index[i] = t
                    class_rooms[i] = classrooms_data[c]

    assert len(times) == I

In [64]:
res2 = course_data.copy()
res2["Time"] = res2.index.map(times)
res2["Classroom"] = res2.index.map(class_rooms)
res2["Time_index"] = res2.index.map(times_index)
res2["Score"] = res2.apply(lambda x: x["Preferences"].index(x["Time_index"]) + 1, axis=1)
res2

,Crs & Sec,Instructor,Preferences,Time,Classroom,Time_index,Score
0,AFR-101-0,Garcia-Rojas C,"[13, 6, 12, 0]",M W F 8:30-9:20,WALL 351,0,4
1,AFR-283-0,Gill-Sadler R,"[4, 8, 7, 6]",M W F 14:30-15:20,WATSON 310,6,4
2,AFR-302-0,Green H,"[12, 4, 7, 5]",M W F 13:30-14:20,CHAM 2164,5,4
3,AFR-495-0,Green H,"[9, 3, 8, 2]",M W F 10:30-11:20,LIB STUDIOD,2,4
4,ANT-101-0,Shuman S,"[4, 2, 1, 9]",T R 8:15-9:30,CHAM 3084,9,4
...,...,...,...,...,...,...,...
339,PSY-300-0,Eiler B,"[11, 13, 3, 5]",M W F 13:30-14:20,WALL 320,5,4
340,PSY-302-A,Smith M,"[13, 7, 10, 11]",T R 12:15-13:30,WALL 106,11,4
341,PSY-302-B,Smith M,"[10, 3, 4, 5]",M W F 13:30-14:20,CHAM 1045,5,4
342,PSY-302-C,Smith M,"[10, 2, 4, 0]",M W F 8:30-9:20,WATSON 109,0,4


In [71]:
res2.to_csv("/Users/admin/Desktop/Codeee/CSC-385-Project/results.csv")

In [65]:
check = res2.groupby("Time")['Instructor'].apply(list).to_dict()
for list_profs in check.values():
    check_set = set()
    for prof in list_profs:
        check_set.add(prof)
    assert len(list_profs) == len(check_set)

In [66]:
res2.groupby("Instructor")['Score'].apply(list).to_dict()

{'Ahrensdorf P': [4, 4, 4],
 'Aldridge D': [4, 4],
 'Alexander C': [4, 4],
 'Anstey M': [3, 4, 4],
 'Bahr H': [4, 4],
 'Bailey I': [4, 4, 4],
 'Baker B': [4, 4, 4],
 'Baugh S': [4, 4],
 'Belloni M': [4, 4, 4],
 'Benson R': [4, 4],
 'Berkey J': [4, 4],
 'Bernd K': [4, 4],
 'Blake H': [4, 4],
 'Blauch D': [4, 4],
 'Bond A': [4, 4],
 'Bory A': [4, 4],
 'Botelho M': [4, 4],
 'Bowles L': [4, 4],
 'Boyd M': [4, 4],
 'Boyer P': [4, 4],
 'Breitenfeld S': [4, 4],
 'Bullock G': [4],
 'Bullock S': [4, 4],
 'Busch K': [4],
 'Campbell M': [4, 4],
 'Campbell S': [4],
 'Ceka B': [4, 4, 4],
 'Chan F': [4, 4],
 'Chapman L': [3, 4, 4],
 'Chillag K': [4, 4],
 'Cools A': [4, 4],
 'Cormier A': [4, 4],
 'Corso-Esquivel J': [4, 4],
 'Crandall B': [3, 4],
 'Crandall R': [4],
 'Crowder-Meyer M': [4, 4, 4],
 'Dietrick J': [4, 4, 4],
 'Dietz V': [3, 4],
 'Duhon A': [4, 4, 4],
 'Eiler B': [4],
 'El Bejjani R': [4, 4],
 'El-Zaatari B': [4, 4],
 'Ewoodzie J': [4],
 'Fackler M': [4, 3],
 'Fernandez R': [4],
 'Fernan

In [67]:
total_score2 = res2.Score.sum()
total_score2

1354

In [68]:
len(res2[res2.Score < 4].index)

22

In [69]:
res2[res2.Score == 2]

,Crs & Sec,Instructor,Preferences,Time,Classroom,Time_index,Score


In [70]:
res2[res2.Score < 2]

,Crs & Sec,Instructor,Preferences,Time,Classroom,Time_index,Score
